# Reliability analysis of the critical head difference

In this example, we will demonstrate how to conduct a reliability analysis of the critical head difference model developed by Sellmeijer. This model is applicable to the piping failure mechanism, which addresses backward internal erosion beneath dikes with predominantly horizontal seepage paths.

### Define model

First, let's import the necessary packages:

In [1]:
from streams import *
import numpy as np

The critical head difference, $H_c$, according to the Sellmeijer's model is described by the following equations:

$F_{resistance}=\eta\cdot \frac{\gamma_{sub,particles}}{\gamma_{water}}\cdot \tan \theta_{sellmeijer,rev}$


$F_{scale}=\frac{d_{70.m}}{\sqrt[3]{\kappa\cdot L}}\cdot\left(\frac{d_{70}}{d_{70.m}}\right)^{0.4}$ and $\kappa = \frac{\nu_{water}}{g}\cdot k$


$F_{geometry}=0.91\cdot \left(\frac{D}{L}\right)^{\frac{0.28}{\left(\frac{D}{L}\right)^{2.8}-1}+0.04}$

$H_c = F_{resistance} \cdot F_{scale} \cdot F_{geometry} \cdot L$

where: <br>
$L$ - seepage length (m) <br>
$D$ - thickness of upper sand layer (m) <br>
$\theta$ - bedding angle ($\circ$) <br>
$d_{70}$ - particle diameter (m) <br>
$k$ - permeability of the upper sand layer (m/s)

In [2]:
def model_sellmeijer(k, L, d70, D):

    RD = 0.725
    RDm = 0.725
    d70m = 2.08e-4
    nu = 1.33e-6
    eta = 0.25
    theta = 37.0

    F_resistance = 1.65*eta*np.tan(theta/180*np.pi)*(RD/RDm)**0.35
    F_scale = d70m/(nu/9.81*k*L)**(1/3)*(d70/d70m)**0.39
    F_geometry = 0.91*(D/L)**(0.28/(((D/L)**2.8)-1)+0.04)

    if D==L:
        F_geometry = 1.0
    
    delta_h_c = F_resistance * F_scale * F_geometry * L
    
    # uncomment next line to print all model evaluations
    # print(str(delta_h_c))

    return delta_h_c

### Perform sensitivity analysis

The goal is to estimate the effect of the input parameters $k$, $L$, $d70$ and $D$ on the critical head difference. 
To achieve this, we conduct a sensitivity analysis, we begin by creating a sensitivity project and defining the model:

In [3]:
project = ReliabilityProject()
project.model = model_sellmeijer

The input and output parameters are retrieved from the model, You will need them to specifiy stochastic variables and define a limit state function.

In [4]:
print("INPUT PARAMETERS")
for input_parameter in project.model.input_parameters:
    print (input_parameter)
    
print("")
print("OUTPUT PARAMETERS")
for output_parameter in project.model.output_parameters:
    print (output_parameter)

INPUT PARAMETERS
k
L
d70
D

OUTPUT PARAMETERS
delta_h_c


We define all the input parameters of the model as log normal variables, all with variation coefficient of 0.25:

In [5]:
project.variables["k"].distribution = DistributionType.log_normal
project.variables["k"].mean = 0.000245598
project.variables["k"].variation = 0.25

project.variables["L"].distribution = DistributionType.log_normal
project.variables["L"].mean = 40.0
project.variables["L"].variation = 0.25

project.variables["d70"].distribution = DistributionType.log_normal
project.variables["d70"].mean = 0.00019
project.variables["d70"].variation = 0.25

project.variables["D"].distribution = DistributionType.log_normal
project.variables["D"].mean = 30.0
project.variables["D"].variation = 0.25


We use the `crude_monte_carlo` method and define the relevant settings: `minimum_samples` and `maximum_samples`.

The reliability analysis is performed using `project.run()`, and the results can be accessed from `project.design_point`.

In [6]:
project.settings.reliability_method = ReliabilityMethod.crude_monte_carlo
project.settings.minimum_samples = 1000
project.settings.maximum_samples = 2000

project.limit_state_function.parameter = project.model.output_parameters[0]
project.limit_state_function.compare_type = CompareType.greater_than
project.limit_state_function.critical_value = 3.0

project.run()


In [7]:
def read_results(dp):

    beta = dp.reliability_index

    print(f"Beta = {beta}")

    pf = StandardNormal.get_q_from_u(beta)
    print(f"Probability of failure = {pf}")

    for alpha in dp.alphas:
        print(f"{alpha.variable.name}: alpha = {alpha.alpha}, x = {alpha.x}")

    if dp.is_converged:
        print(f"Converged (convergence = {dp.convergence} < {project.settings.variation_coefficient})")
    else:
        print(f"Not converged (convergence = {dp.convergence} > {project.settings.variation_coefficient})")
        
    print(f"Model runs = {dp.total_model_runs}")
    

    
read_results(project.design_point)

Beta = 1.0849745343197774
Probability of failure = 0.13896646712852412
k: alpha = 0.32935939266495734, x = 0.00021819684882517845
L: alpha = -0.8438921637577063, x = 48.61864825602459
d70: alpha = -0.3905332385704941, x = 0.00020459656129760324
D: alpha = 0.16386639674853695, x = 27.857696480798534
Converged (convergence = 0.004469942082884176 < 0.05)
Model runs = 25
